## Atomistic Molecular Dynamics: Bulk Liquids - Mixtures

For our final system in this session, we will consider a system that features a mixture of water and alkanes.

### Build the system

Again, we will use mBuild to construct our system.

First, we will import mBuild, as well as specify a filter for some warnings that can often clutter the output.

We will also import classes for `Alkane` and `H2O` (these are the exact same class definitions as we created in the previous notebooks).

In [ ]:
import mbuild as mb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from utils import Alkane, H2O

Now we'll define a class for the mixture of water and alkanes. In standard mBuild style, we want to make sure any chemistry modifications we would like to make can be achieved by passing arguments upon class instantiation. For this class some of the chemical modifications we might want to make are:

  - Chain length of the alkanes
  - Number of chains to add to the box
  - Number of water molecules to add to the box
  - Density at which to pack the molecules

In [ ]:
class MixtureBox(mb.Compound):
    """An box of linear alkane chains and water."""
    def __init__(self, chain_length, n_chains, n_water, density):
        """Initialize an AlkaneBox Compound.

        Parameters
        ----------
        chain_length : int
            Length of the alkane chains (in number of carbons)
        n_chains : int
            Number of chains to place in the box
        n_water : int
            Number of water molecules to place in the box
        density : float
            Density (in kg/m^3) at which the system should be created
        """
        super(MixtureBox, self).__init__()
        
        # Create alkane chain prototype using the class we've imported
        chain = Alkane(chain_length=chain_length)
        
        # Generate a more relaxed structure for the alkane
        chain.energy_minimization()
        
        # Create water prototype using the class we've imported
        h2o = H2O()
        
        # Fill a box with alkanes and water at a user-defined density
        mixture_box = mb.fill_box(compound=[chain, h2o], n_compounds=[n_chains, n_water],
                                  density=density)
        
        # Rename all chains to 'Alkane' and all the waters to 'Water'
        # This speeds up the atom-typing process
        for child in mixture_box.children:
            if child.n_particles > 3:
                child.name = 'Alkane'
            else:
                child.name = 'Water'
        self.add(mixture_box)

Let's create some systems. First we'll define the chain length for the alkane chains we want to create, along with the number of chains and number of water molecules we want to add to the box.

In [ ]:
chain_length = 6
n_chains = 150
n_water = 200

While we will be performing NPT simulations (so the density will be changing), it is a good idea to start the system off at a reasonable density. For a simple estimation we will use:

\begin{align}
\rho_{mix} = x_{alkane} \times \rho_{alkane} + x_{water} \times \rho_{water}
\end{align}

In [ ]:
mass_water = 18.01528 * n_water
mass_alkane = (13.019 * (chain_length - 2) + 2 * 14.027) * n_chains
total_mass = mass_water + mass_alkane

alkane_densities = {5: 626, 6: 655, 7: 684, 8: 703, 9: 718, 10: 730}

density = (mass_alkane / total_mass) * alkane_densities[chain_length] + \
          (mass_water / total_mass) * 1000

Now let's actually create the mBuild `Compound` based on the parameters we've defined.

In [ ]:
mixture_box = MixtureBox(chain_length=chain_length, n_chains=n_chains, n_water=n_water,
                         density=density)

Now we'll visualize the `Compound` we've just created.

In [ ]:
mixture_box.visualize()

Feel free to play around with the system parameters by re-running the four code blocks above.

Once you have a system that you like, we will again be saving to the two GROMACS files. First we'll write out our GRO file.

In [ ]:
mixture_box.save('mixture.gro', overwrite=True, residues=['Alkane', 'Water'])

Now we'll write out our TOP file.

In [ ]:
mixture_box.save('mixture.top', forcefield_files='utils/spce-alkane.xml', overwrite=True,
                 residues=['Alkane', 'Water'])

Note that for the `forcefield_files` argument we used a file titled `spce-alkane.xml`. This is a file that has been created that combines the OPLS parameters for alkanes with the SPC/E parameters for water. Let's take a quick look at this:

In [ ]:
!cat utils/spce-alkane.xml

### Run the simulation

We'll now perform our simulation, again in the NPT ensemble.

Execute the cell below to call `grompp` followed by `mdrun`.

In [ ]:
!gmx grompp -v -f utils/npt-mix.mdp -c mixture.gro -p mixture.top -o npt-mix
!gmx mdrun -v -s npt-mix.tpr -o -x -deffnm npt-mix

Now we'll view the trajectory with NGLview. Alternatively, you can pull the trajectory up in VMD if you would like more control over the representation.

**NOTE:** If using VMD, input the following command from a terminal session (assuming you are in the same directory where the output files are located):
`vmd mixture.mol2 npt-mix.xtc`

In [ ]:
import nglview as nv
import mdtraj as md

mixture_box.save('mixture.mol2', overwrite=True)
t = md.load("npt-eq-mix.xtc", top="mixture.mol2")
nv.show_mdtraj(t)

We'll again use `gmx energy` to extract some information on the system. Execute the two cells below to extract this information and plot it using `matplotlib`.

In [ ]:
!echo 11 12 13 18 19\\n0 | gmx energy -f npt-eq-mix.edr -o npt-eq-mix.xvg

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('npt-eq-mix.xvg', skiprows=27)

# Strip out the very beginning of the run
data = data[10:]

fig, ax = plt.subplots(5, 1)

properties = ['Total energy, kJ/mol', 'Temperature, K', 'Pressure, atm',
              'Volume, nm^3', 'Density, kg/m^3']

for i, sub_ax in enumerate(ax):
    sub_ax.plot(data[:, 0], data[:, i + 1])
    sub_ax.set_title(properties[i])
    sub_ax.set_ylabel(properties[i])
    
fig.subplots_adjust(hspace=1.0)
fig.set_size_inches(4, 12)
plt.show()